In [1]:
library(Seurat)
library(Matrix)

In [2]:
data_path_ref='/home/khrovatin/retinal/data/counts/'
data_path='/home/khrovatin/retinal/data/human/'
data_path_shared='/share/LBI_share/public/retina/human\ snRNAseq\ data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix'

In [3]:
seurat.obj_int<-readRDS(paste(data_path_ref,'scaled_integrated.rds',sep=''))

ERROR: Error: cannot allocate vector of size 1.5 Gb


In [ ]:
GetAssayData(object = seurat.obj_nb, slot = "scale.data")

In [6]:
expression_matrix <- Read10X(data.dir = data_path_shared)
seurat.obj_h = CreateSeuratObject(counts = expression_matrix)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [7]:
seurat_both=list()
seurat_both[['ref']]<-seurat.obj_int
seurat_both[['sample']]<-seurat.obj_h

In [8]:
genes_ref_var=rownames(GetAssayData(object = seurat_both[['ref']], slot = "scale.data"))

In [9]:
genes_ref_var[1:10]

[1] "NID1"         "GNG4"         "IRF2BP2"      "MSTRG.100"    "MSTRG.103"   
 [6] "C1H1orf198"   "CAPN9-p"      "AGT"          "LOC102130133" "MGST3"

In [11]:
genes_sample<-rownames(seurat_both[['sample']])

In [13]:
genes_integration<-intersect(genes_sample,genes_ref_var)

In [15]:
length(genes_integration)

[1] 2203

In [16]:
dim(seurat.obj_h)

[1] 32738 11245

In [17]:
m<-matrix(c(0,0,1,0), nrow = 2, ncol = 2)

In [18]:
m

0,1
0,0


In [39]:
col_not_null<-colSums(m!=0)
keep_cells<-col_not_null>0
m[,keep_cells]

[1] 1 0

In [35]:
m

[1] 0 1

In [8]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from collections import OrderedDict
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
import pandas as pd
import h5py

import scipy.io


from sklearn.linear_model import LogisticRegression

In [2]:
import sklearn
sklearn.__version__

'0.22'

In [19]:
from platform import python_version

print(python_version())

3.7.4


In [15]:
m1=np.random.rand(10,2)
m2=np.random.rand(10,2)
c1=np.random.randint(1,5,10)
c2=np.random.randint(1,5,10)

In [26]:


classifier = KNeighborsClassifier(weights='distance',n_jobs=4)
classifier.fit(m1,c1)
prediction=classifier.predict(m2)
truth=c2
labels=[1,2,3,4]
precision, recall, fscore, support = score(y_true=truth, y_pred=prediction,labels=labels)
precision_all, recall_all, fscore_all, support_all=score(
    y_true=truth, y_pred=prediction,labels=labels,average='weighted')
print(pd.DataFrame({'class':np.append(classifier.classes_,'weighted average'),'precision':np.append(precision,precision_all),'recall':np.append(recall,recall_all),
                    'fscore':np.append(fscore,fscore_all),
                  'N true':np.append(support,support_all)}))
prediction_p=classifier.predict_proba(m2)
print('ROC AUC score (weighted average)',roc_auc_score(y_true=truth, y_score=prediction_p,multi_class='ovr', average='weighted'))

              class  precision  recall  fscore N true
0                 1   0.333333     1.0    0.50      2
1                 2   0.333333     1.0    0.50      1
2                 3   0.000000     0.0    0.00      4
3                 4   0.000000     0.0    0.00      3
4  weighted average   0.100000     0.3    0.15   None
ROC AUC score (weighted average) 0.6321428571428571


/home/khrovatin/miniconda3/envs/env1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
classifier.predict_proba(m2)

array([[0.        , 0.17735374, 0.2734962 , 0.54915006],
       [0.        , 0.46258014, 0.38953547, 0.14788439],
       [0.        , 0.4298138 , 0.14603594, 0.42415026],
       [0.        , 0.41271411, 0.43443305, 0.15285284],
       [0.22942029, 0.53123731, 0.        , 0.2393424 ],
       [0.        , 0.46811366, 0.        , 0.53188634],
       [0.1793045 , 0.58336388, 0.        , 0.23733162],
       [0.17445508, 0.25585949, 0.        , 0.56968543],
       [0.        , 0.52151673, 0.22719646, 0.25128681],
       [0.25058095, 0.29896958, 0.23762105, 0.21282843]])

In [18]:
classifier.classes_

array([1, 2, 3, 4])

In [8]:
m=sp.io.mmread('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/matrix.mtx.gz')

In [13]:
m=pd.DataFrame.sparse.from_spmatrix(m).T

AttributeError: 'DataFrame' object has no attribute 'tocsc'

In [22]:
m.columns=pd.read_table('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/features.tsv.gz',header=None)[1]

In [25]:
m.index=pd.read_table('/share/LBI_share/public/retina/human snRNAseq data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix/barcodes.tsv.gz',header=None)[0]

In [26]:
m

1,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,AL627309.1,RP11-34P13.14,RP11-34P13.9,AP006222.2,RP4-669L17.10,...,KIR3DL2,AL590523.1,CT476828.1,PNRC2,SRSF10,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1
0,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAATAACC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTCCTGCG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTCTAACC-1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCACAAGAAACT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCACAAGAGTAT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCACTCGAA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TTTGTTGTCATAAGGA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
TTTGTTGTCATGCATG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data_path='/home/khrovatin/retinal/data/counts/'
with h5py.File(data_path+'normalisedlog.h5','r') as file:    
    data_norm=pd.DataFrame(file.get('ref/matrix')[:200,:])
    data_norm.columns=[name.decode() for name in file.get('ref/rownames')]
    data_norm.index=[name.decode() for name in file.get('ref/colnames')[:200]]
    
    data_norm_h=pd.DataFrame(file.get('00012_NeuNM/matrix')[:200,:])
    data_norm_h.columns=[name.decode() for name in file.get('00012_NeuNM/rownames')]
    data_norm_h.index=[name.decode() for name in file.get('00012_NeuNM/colnames')[:200]]

In [7]:
col_data=pd.read_table(data_path+'passedQC_cellData.tsv',index_col=0)
col_by_region=col_data.groupby('region')
fov_cells=col_by_region.get_group('fov').index
per_cells=col_by_region.get_group('per').index

In [23]:
log_reg=LogisticRegression(penalty='l1',C=0.8,random_state=0,solver='saga',n_jobs=4).fit(
    data_norm, col_data.iloc[np.random.randint(1,150000,200),:]['cell_type'])

/home/khrovatin/miniconda3/envs/env1/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [58]:
def make_log_regression(data1,data2,col_data,label='cell_type',
                        logreg={'penalty':'l1','C':0.8,'random_state':0,'solver':'saga','n_jobs':8}):
    log_reg=LogisticRegression(**logreg).fit(data1, col_data.loc[data1.index,label])
    print('** Statistics train **')
    evaluate_classifier(classifier=log_reg,data=data1,col_data=col_data,label=label)
    print('** Statistics test **')
    evaluate_classifier(classifier=log_reg,data=data2,col_data=col_data,label=label)
    return log_reg
    
    
def evaluate_classifier(classifier,data,col_data,label):
    prediction=classifier.predict(data)
    truth=col_data.loc[data.index,label].values
    labels=list(set(col_data.loc[data.index,label]))
    precision, recall, fscore, support = score(y_true=truth, y_pred=prediction,labels=labels)
    precision_all, recall_all, fscore_all, support_all=score(
        y_true=truth, y_pred=prediction,labels=labels,average='weighted')
    print(pd.DataFrame({'class':np.append(classifier.classes_,
               'weighted average'),'precision':np.append(precision,precision_all),
             'recall':np.append(recall,recall_all),
             'fscore':np.append(fscore,fscore_all),
            'N true':np.append(support,support_all)}))
    prediction_p=classifier.predict_proba(data)
    print('ROC AUC score (weighted average)',roc_auc_score(y_true=truth, y_score=prediction_p,
                                                           multi_class='ovr', average='weighted'))


In [60]:
classifier=make_log_regression(data_norm,data_norm,pd.DataFrame(['a','b','c','d']*50,index=data_norm.index),label=0)

** Statistics train **
              class  precision  recall  fscore N true
0                 a        1.0     1.0     1.0     50
1                 b        1.0     1.0     1.0     50
2                 c        1.0     1.0     1.0     50
3                 d        1.0     1.0     1.0     50
4  weighted average        1.0     1.0     1.0   None
ROC AUC score (weighted average) 1.0
** Statistics test **
              class  precision  recall  fscore N true
0                 a        1.0     1.0     1.0     50
1                 b        1.0     1.0     1.0     50
2                 c        1.0     1.0     1.0     50
3                 d        1.0     1.0     1.0     50
4  weighted average        1.0     1.0     1.0   None
ROC AUC score (weighted average) 1.0


/home/khrovatin/miniconda3/envs/env1/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [43]:
#data_norm,data_norm,pd.DataFrame(['a','b']*100,index=data_norm.index),label=0
cd=pd.DataFrame(['a','b']*100,index=data_norm.index)

In [57]:
label=0
truth=cd.loc[data_norm.index,label].values
print(truth)
labels=list(set(cd.loc[data_norm.index,label]))
labels
prediction_p=classifier.predict_proba(data_norm)
prediction_p
#print('ROC AUC score (weighted average)',roc_auc_score(y_true=truth, y_score=prediction_p,
#                                                           multi_class='ovr', average='weighted'))


['a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'
 'a' 'b']


array([[0.93753288, 0.06246712],
       [0.02803859, 0.97196141],
       [0.9527823 , 0.0472177 ],
       [0.0424855 , 0.9575145 ],
       [0.92324171, 0.07675829],
       [0.03089597, 0.96910403],
       [0.95169514, 0.04830486],
       [0.0848331 , 0.9151669 ],
       [0.97570842, 0.02429158],
       [0.03544931, 0.96455069],
       [0.97084811, 0.02915189],
       [0.06640527, 0.93359473],
       [0.9688183 , 0.0311817 ],
       [0.05811448, 0.94188552],
       [0.96134202, 0.03865798],
       [0.01936887, 0.98063113],
       [0.97305821, 0.02694179],
       [0.04979218, 0.95020782],
       [0.92294938, 0.07705062],
       [0.03569416, 0.96430584],
       [0.9517601 , 0.0482399 ],
       [0.07005032, 0.92994968],
       [0.97736309, 0.02263691],
       [0.06360856, 0.93639144],
       [0.96569709, 0.03430291],
       [0.06834536, 0.93165464],
       [0.98219738, 0.01780262],
       [0.01794815, 0.98205185],
       [0.94188285, 0.05811715],
       [0.07752903, 0.92247097],
       [0.

In [34]:
pd.DataFrame(['a','b']*100,index=data_norm.index)

,0
M1CD90PNA_S1_AACTGGTAGTGGTCCC.1,a
M1CD90PNA_S1_AAGACCTTCTCTTATG.1,b
M1CD90PNA_S1_AAGGAGCAGTAGGCCA.1,a
M1CD90PNA_S1_AAGGTTCAGAGTCGGT.1,b
M1CD90PNA_S1_AAGTCTGCACGCCAGT.1,a
...,...
M1PerCD73S1_AAACGGGAGGCCCTCA.1,b
M1PerCD73S1_AAACGGGAGGCTATCT.1,a
M1PerCD73S1_AAAGATGTCAGCGACC.1,b
M1PerCD73S1_AAAGCAAAGGACAGAA.1,a
